In [89]:
from typing import TypedDict, Annotated, List, Literal, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END, MessagesState
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
import PyPDF2 as pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate

In [90]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [91]:
llm = ChatGroq(model = "llama-3.3-70b-versatile")

In [92]:
class SupervisorState(MessagesState):
    """State for the multi-agent"""
    next_agent: str = "ناظر"
    research_data: str = ""
    summarize_data: str = ""
    final_report: str = ""
    task_complete: bool = False
    current_task: str = ""

In [93]:
def create_supervisor_chain():
    """Creates the supervisor decision chain"""
    supervisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """تو در نقش یک ناظر هستی که یک تیم از ایجنت ها را مدیریت می کند:

1. محقق - اطلاعات و داده ها را جمع آوری می کند
2. خلاصه گر: یک متن را خلاصه می کند
3. نویسنده: گزارشی از متن ارائه می کند

بر اساس حالت فعلی و گفتگو، تصمیم بگیر کدام ایجنت باید در حالت بعدی مورد استفاده قرار گیرد.
اگر وظیفه انجام شد، با کلمه "انجام شد" پاسخ بده.

حالت فعلی:
- آیا داده تحقیقاتی دارد: {has_research}
- آیا داده خلاصه دارد: {has_summarize}
- آیا داده گزارش دارد: {has_report}

فقط نام ایجنت، یعنی "محقق" یا "خلاصه گر"، یا "نویسنده" یا "انجام شد" را در پاسخ ارائه کن.
"""),
        ("human", "{task}")
    ])

    return supervisor_prompt | llm

In [94]:
def supervisor_agent(state: SupervisorState) -> Dict:
    messages = state.get("messages", [])
    current_task = state.get("current_task", "")

    if not current_task:
        human_msgs = [m for m in messages if isinstance(m, HumanMessage)]
        if human_msgs:
            current_task = human_msgs[-1].content.strip()
        else:
            current_task = "هیچ وظیفه ای وجود ندارد"

    has_research = bool(state.get("research_data"))
    has_summarize = bool(state.get("summarize_data"))
    has_report = bool(state.get("final_report"))

    chain = create_supervisor_chain()
    decision = chain.invoke({
        "task": current_task,
        "has_research": has_research,
        "has_summarize": has_summarize,
        "has_report": has_report
    })

    decision_text = decision.content.strip().lower()
    print(f"Supervisor decision: {decision_text}")

    if "انجام شد" in decision_text or has_report:
        next_agent = "__end__"
        supervisor_msg = "ناظر: تمام وظایف کامل شد! کار تیمی عالی بود."
    elif "خلاصه گر" in decision_text or (has_research and not has_summarize):
        next_agent = "خلاصه گر"
        supervisor_msg = "ناظر: تحقیق انجام شد. زمان خلاصه کردن متن است."
    elif "نویسنده" in decision_text or (has_summarize and not has_report):
        next_agent = "نویسنده"
        supervisor_msg = "ناظر: خلاصه سازی تمام شد. در حال نگارش گزارش نهایی."
    else:
        next_agent = "محقق"
        supervisor_msg = "ناظر: شروع با محقق برای جمع‌آوری اطلاعات."

    return {
        "messages": state["messages"] + [AIMessage(content=supervisor_msg)],
        "next_agent": next_agent,
        "current_task": current_task  
    }


In [95]:
def researcher_agent(state: SupervisorState) -> Dict:
    """Researcher uses Groq to gather information"""
    
    task = state.get("current_task", "تحقیق در مورد موضوع مشخص شده")

    def get_texts(pdf_path):
        with open(pdf_path, "rb") as file:
            reader = pdf.PdfReader(file)
            texts = ""
            for page in reader.pages:
                extracted = page.extract_text()
                if extracted:
                    texts += extracted
            return texts
    
    def get_chunks(texts):
        splitter = RecursiveCharacterTextSplitter(
            separators = ["\n\n", "\n", " "],
            chunk_size = 1500,
            chunk_overlap = 500
        )
        chunks = splitter.split_text(texts)
        return chunks

    texts = get_texts("C:/Users/Lenovo/Desktop/New folder (6)/rooze.pdf")
    chunks = get_chunks(texts)

    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_texts(chunks, embedding = embeddings)
    retriever = db.as_retriever(k = 3)
    prompt = ChatPromptTemplate.from_template(
        """
        شما یک متخصص خبره در حوزه پاسخ به سوالات دینی به خصوص پاسخ به سوالات درمورد روزه هستید و هدف شما ارائه یک پاسخ علمی و ساختارمند با جزئیات کامل به سوال مطرح شده فقط با توجه به متن ارائه شده به شما است.

        سوال: {input}

        دستورالعمل ها:
        1. پاسخ باید با جمله ای آغاز شود که پیش زمینه و اهمیت موضوع را مشخص کند. مثال: "در اسلام، روزه جایگاه ویژه ای دارد".
        2. پاسخ باید شامل بخش های زیر باشد:
        تحلیل موضوع: بررسی دقیق موضوع و جزئیات فنی
        زیر موضوع ها: ارائه نکات جزئی مرتبط با هر زیر موضوع
        3. نتیجه گیری: خلاصه نکات کلیدی

        پاسخ باید دقیق، ساختارمند، مرتبط و بدون مطالب غیر ضروری باشد. از عبارات کلی و مبهم پرهیز کنید.

        متن زیر را به دقت مطالعه و اطلاعات مرتبط با موضوع سوال استخراج شود:

        متن: {context}

        طول پاسخ 1000 کلمه باشد.

        برای هر بخش پاسخ از جدا کننده "---" استفاده شود تا بخش ها کاملا تفکیک شده باشند.

        لطفا پاسخی تولید کنید که تمام دستورالعمل های بالا را رعایت کند و تمرکز اصلی آن روی موضوع سوال مطرح شده باشد.
        """
    )
    documents_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, documents_chain)
    
    researcher_response = retrieval_chain.invoke({"input": task})
    research_data = researcher_response["answer"]
    
    agent_message = f"""محقق: من تحقیق را در مورد '{task}' کامل کردم.\n\nنکات کلیدی پیدا شده:\n{research_data}"""

    return {
        #"messages": [AIMessage(content = agent_message)],
        "messages": state["messages"] + [AIMessage(content = agent_message)],
        "research_data": research_data,
        "next_agent": "ناظر"
    }

In [96]:
def summarize_agent(state: SupervisorState) -> Dict:
    """Summarizer uses Groq to summarize the research"""
    research_data = state.get("research_data", "") 
    task = state.get("current_task", "") 
    
    doc = [Document(page_content = research_data)]

    prompt = PromptTemplate(
        template = """
شما یک متخصص خبره در زمینه خلاصه سازی متون دینی به خصوص متون مربوط به روزه هستید و وظیفه شما این است که متن زیر را به دقت و به طور خلاصه بیان کنید. شما باید تمامی اطلاعات کلیدی را استخراج کرده و خلاصه ای واضح، ساختارمند و مفهومی ارائه دهید.

دستورالعمل ها:
1. خلاصه باید با جمله ای آغاز شود که زمینه و اهمیت اطلاعات مورد نظر را مشخص کند. مثال: "این متن در مورد موضوع روزه توضیح می دهد که در اسلام اهمیت بالایی دارد."
2. خلاصه باید به شکل زیر ارائه شود:
مقدمه: توضیح کوتاه درباره محتوای اصلی متن.
نکات کلیدی: استخراج نکات مهم و اطلاعات اصلی.
نتیجه گیری: خلاصه ای از نتیجه گیری ها یا پیام اصلی متن.
3. خلاصه باید بدون حشو و اضافات باشد و تمام نکات ضروری به طور دقیق بیان شود. از عبارات غیرضروری پراکنده پرهیز کنید.
4. متن زیر را با دقت مطالعه کرده و اطلاعات مهم آن را در قالب خلاصه استخراج کنید.
متن: {text}

طول متن خلاصه شده باید 500 کلمه باشد. 

برای تفکیک بخش های مختلف خلاصه از جدا کننده "---" استفاده کنید. این جدا کننده ها باید در ابتدا و انتهای هر بخش (مقدمه، نکات کلیدی، نتیجه گیری) قرار گیرد.
""",
        input_variables = ["text"]
    )

    chain = load_summarize_chain(
        llm,
        chain_type = "stuff",
        prompt = prompt,
        verbose = False
    )

    response = chain.run(doc)
    
    agent_message = f"خلاصه گر: من خلاصه سازی متن را کامل کردم.\n\nمتن خلاصه شده:\n{response}"

    return {
        #"messages": [AIMessage(content = agent_message)],
        "messages": state["messages"] + [AIMessage(content = agent_message)],
        "summarize_data": response,
        "next_agent": "ناظر"
    }

In [97]:
def writer_agent(state: SupervisorState) -> Dict:
    """Writer uses Groq to create final report"""
    research_data = state.get("research_data", "")
    summarize_data = state.get("summarize_data", "")
    task = state.get("current_task")

    writing_prompt = f"""
شما یک تحلیلگر خبره در زمینه تحلیل محتوا هستید و وظیفه شما این است که بر اساس وظیفه مشخص شده، گزارشی جامع و کلی از دو متن (متن اصلی و متن خلاصه شده) ارائه دهید. هدف این گزارش این است که بتوانید یک دیدگاه کلی و جامع از هر دو متن و نحوه انتقال اطلاعات اصلی آن ها ارائه دهید. این گزارش باید به گونه ای باشد که به جزئیات اصلی هر دو متن پرداخته شود.

دستور العمل ها:

1. گزارش باید به طور واضح و ساختارمند به ارائه یک دیدگاه کلی پرداخته و از عبارت های غیر صروری و یا پیچیده پرهیز کند.
2. از متن های زیر برای ارائه دیدگاه کلی و تحلیلی استفاده کنید. باید اطلاعات کلیدی هر متن را به طور فشرده استخراج کنید و در قالب یک گزارش جامع بیان کنید.

وظیفه: {task}

متن اصلی: {research_data}

متن خلاصه شده: {summarize_data}

طول این گزارش باید 500 کلمه باشد.

برای تفکیک بخش های مختلف گزارش از جدا کننده "---" استفاده کنید. این جدا کننده ها باید در ابتدا و انتهای هر بخش قرار گیرد.

لطفا گزارشی جامع و کلی از دو متن بر اساس وظیقه مطرح شده ارائه دهید که تمامی دستورالعمل های بیان شده را رعایت کند.
"""
    
    report_response = llm.invoke([HumanMessage(content = writing_prompt)])
    report = report_response.content

    final_report = f"""
    گزارش نهایی
{'=' * 50}
موضوع: {task}
{'=' * 50}

{report}
"""
    
    return {
        #"messages": [AIMessage(content = f"نویسنده: گزارش کامل شد! سند کامل در زیر قابل دیدن است.")],
        "messages": state["messages"] + [AIMessage(content = "نویسنده: گزارش کامل شد! سند کامل در زیر قابل دیدن است")],
        "final_report": final_report,
        "next_agent": "ناظر",
        "task_complete": True
    }

In [98]:
def router(state: SupervisorState):
    next_agent = state.get("next_agent", "ناظر")
    
    if next_agent in ["__end__", "end"] or state.get("task_complete", False):
        return END
    
    if next_agent in ["ناظر", "محقق", "خلاصه گر", "نویسنده"]:
        return next_agent
    
    return "ناظر"

In [99]:
workflow = StateGraph(SupervisorState)

workflow.add_node("ناظر", supervisor_agent)
workflow.add_node("محقق", researcher_agent)
workflow.add_node("خلاصه گر", summarize_agent)
workflow.add_node("نویسنده", writer_agent)

workflow.set_entry_point("ناظر")

for node in ["ناظر", "محقق", "خلاصه گر", "نویسنده"]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "ناظر": "ناظر",
            "محقق": "محقق",
            "خلاصه گر": "خلاصه گر",
            "نویسنده": "نویسنده",
            "__end__": END
        }
    )

graph = workflow.compile()

In [79]:
response = graph.invoke({
    "messages": [HumanMessage(content="روزه برای چه کسانی واجب نیست؟")]
})

Supervisor decision: محقق
Supervisor decision: خلاصه گر
Supervisor decision: نویسنده


In [80]:
response

{'messages': [HumanMessage(content='روزه برای چه کسانی واجب نیست؟', additional_kwargs={}, response_metadata={}, id='9ee6949f-59b8-4420-bda4-c6b7f1b67429'),
  AIMessage(content='ناظر: شروع با محقق برای جمع\u200cآوری اطلاعات.', additional_kwargs={}, response_metadata={}, id='72cd9e3e-6ace-41be-99f4-d3dfd00332d1'),
  AIMessage(content="محقق: من تحقیق را در مورد 'روزه برای چه کسانی واجب نیست؟' کامل کردم.\n\nنکات کلیدی پیدا شده:\nدر اسلام، روزه جایگاه ویژه\u200cای دارد و یکی از ارکان مهم دین است. روزه\u200cداری در ماه رمضان برای 모든 مسلمانان واجب است، مگر برای برخی که به دلیل شرایط خاصی از این تکلیف معاف هستند. در این پاسخ، ما به بررسی کسانی که روزه برای họ واجب نیست، می\u200cپردازیم.\n\n---\n### تحلیل موضوع\n\nروزه برای مسلمانانی که دارای شرایط خاصی هستند، واجب نیست. این شرایط شامل بیماری\u200cهای شدید، سفر، بارداری، شیردهی، و کهولت سن می\u200cشود. در این بخش، به تحلیل هر یک از این موضوعات می\u200cپردازیم.\n\n#### زیر موضوع ها\n\n- **بیماری\u200cهای شدید**: اگر شخصی به بیماری شدیدی مبتلا با

In [81]:
print(response)

{'messages': [HumanMessage(content='روزه برای چه کسانی واجب نیست؟', additional_kwargs={}, response_metadata={}, id='9ee6949f-59b8-4420-bda4-c6b7f1b67429'), AIMessage(content='ناظر: شروع با محقق برای جمع\u200cآوری اطلاعات.', additional_kwargs={}, response_metadata={}, id='72cd9e3e-6ace-41be-99f4-d3dfd00332d1'), AIMessage(content="محقق: من تحقیق را در مورد 'روزه برای چه کسانی واجب نیست؟' کامل کردم.\n\nنکات کلیدی پیدا شده:\nدر اسلام، روزه جایگاه ویژه\u200cای دارد و یکی از ارکان مهم دین است. روزه\u200cداری در ماه رمضان برای 모든 مسلمانان واجب است، مگر برای برخی که به دلیل شرایط خاصی از این تکلیف معاف هستند. در این پاسخ، ما به بررسی کسانی که روزه برای họ واجب نیست، می\u200cپردازیم.\n\n---\n### تحلیل موضوع\n\nروزه برای مسلمانانی که دارای شرایط خاصی هستند، واجب نیست. این شرایط شامل بیماری\u200cهای شدید، سفر، بارداری، شیردهی، و کهولت سن می\u200cشود. در این بخش، به تحلیل هر یک از این موضوعات می\u200cپردازیم.\n\n#### زیر موضوع ها\n\n- **بیماری\u200cهای شدید**: اگر شخصی به بیماری شدیدی مبتلا باشد ک

In [82]:
researcher_output = response['messages'][2].content
summarizer_output = response['messages'][4].content
writer_output = response['messages'][6].content
final_report = response['final_report']

print("Researcher Output:\n", researcher_output)
print("Summarizer Output:\n", summarizer_output)
print("Writer Output:\n", writer_output)
print(final_report)

Researcher Output:
 محقق: من تحقیق را در مورد 'روزه برای چه کسانی واجب نیست؟' کامل کردم.

نکات کلیدی پیدا شده:
در اسلام، روزه جایگاه ویژه‌ای دارد و یکی از ارکان مهم دین است. روزه‌داری در ماه رمضان برای 모든 مسلمانان واجب است، مگر برای برخی که به دلیل شرایط خاصی از این تکلیف معاف هستند. در این پاسخ، ما به بررسی کسانی که روزه برای họ واجب نیست، می‌پردازیم.

---
### تحلیل موضوع

روزه برای مسلمانانی که دارای شرایط خاصی هستند، واجب نیست. این شرایط شامل بیماری‌های شدید، سفر، بارداری، شیردهی، و کهولت سن می‌شود. در این بخش، به تحلیل هر یک از این موضوعات می‌پردازیم.

#### زیر موضوع ها

- **بیماری‌های شدید**: اگر شخصی به بیماری شدیدی مبتلا باشد که روزه‌داری برای او ضرر جانی داشته باشد، روزه برای او واجب نیست. این شخص باید روزه‌های از دست رفته را بعداً قضا کند.
- **سفر**: مسافران در ماه رمضان می‌توانند روزه خود را به تأخیر اندازند، اما باید بعداً قضا کنند. اگر سفر طولانی‌مدت باشد، باید روزه را قضا کنند و در صورت سهل‌انگاری بدون عذر، کفاره تأخیر نیز بر آنها واجب است.
- **بارداری و شیردهی**: زنان بار